### Clash Royale card balance analysis

This notebook provides a basic game balance analysis tutorial using a Kaggle dataset of Clash Royale card stats. Click "copy to drive" to edit and run the code.

**Learning goals:** Using the Pandas package to load and manipulate datasets. Using basic linear regression from Scikit Learn for game balance analysis and design.

**Before starting:** Make sure you've played Clash Royale to at least the second arena and understand the basic game mechanics. Also watch Stefan Engblom's [GDC talk](https://www.youtube.com/watch?v=bHLQQh8Ctu4) on balancing Clash Royale.

**How to use this notebook:** Progress cell-by-cell, reading the text and running the code. Click on the small triangles to expand the sections. Some cells give you exercises. Think of those as a puzzle game: Try to solve the exercises without showing the hints and solutions, which are hidden by default. Add a new code cell below each exercise for your attempt. You may copy-paste code from earlier cells to get started.

This notebook is part of Aalto University's Game Analysis course, inspired by the GDC talk above and this [Hearthstone data science tutorial](https://www.kaggle.com/ismailgungor/data-science-tutorial-with-hearthstone-cards)

First, the standard imports (Numpy and PyPlot) plus Pandas for dataset utilities.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd  #.csv file utils etc.

Fetch the dataset from the course's Github, if not already fetched (avoid duplicates if this code is run multiple times). Note that we use the "!" prefix to use the Linux wget command line utility.

In [ ]:
#import file path utilities
from os import path
#check if the dataset already exists
if not path.exists("dataset.csv"):
  #fetch the dataset from the url.
  !wget -O dataset.csv https://raw.githubusercontent.com/PerttuHamalainen/GameAnalysis/master/datasets/clash_wiki_dataset.csv


Create a Pandas DataFrame object from the dataset and show the first few rows using the DataFrame's head() utility.

Note that you can also **interactively browse the dataset**, change the sorting order etc. by clicking on the folder icon on the left to open the Colab file browser, and then double-clicking on "dataset.csv". The ".csv" format is a common human-readable and portable way to store various tabular data.

In [ ]:
data = pd.read_csv("dataset.csv",encoding='iso-8859-1')
data.head()

We see that the data is not super clean: There are many card types and for some cards, some values are NaN:s (which here signal that the value is not defined). There are also some other exceptions such as the damage per second, which is NaN for spells and numeric for other cards, except Inferno tower and Inferno dragon, where it is given as a range of values. To help dealing with this, we first define a helper function for getting a clean subset where rows containing non-numeric values are removed.


In [ ]:
#Returns a new Pandas dataframe consisting of only the given columns.
#Rows with non-numeric values are removed.
#The columns argument can be a single column such as "Cost"
#or a list such as ["Cost","Damage"]
#TODO: This code is ugly, figure out a more concise way.
def getCleanSubset(data,columns):
  subset=data[columns].copy()
  #Also add the card names for easier visualization
  subset["Card"]=data["Card"]
  #remove rows with missing values
  subset=subset.dropna(axis="index")
  #remove rows with something that cannot be expressed as a number
  if not isinstance(columns,list):
    #if the columns is not a list (i.e., it is a single column),
    #wrap it into a list for the loop below
    columns=[columns]
  for column in columns:
    #Remove rows that cannot be converted to a numeric value
    subset=subset[pd.to_numeric(subset[column], errors='coerce').notnull()]
    #Convert to numeric value
    subset[column]=pd.to_numeric(subset[column], errors='coerce')
  return subset

#Test the function
print(getCleanSubset(data,["Cost","Damage per second"]))

### Analyzing cost and benefit of cards using scatter plots and correlation

In Clash Royale, the player has a deck of 8 cards, and deploying each card costs elixir, which is only slowly replenished. The key to winning is investing elixir smartly, getting maximum benefits for each drop of elixir spent. Thus, cards with a high elixir cost are typically more powerful.

The basic problem of balancing the card stats is to keep the cost/benefit ratios of the cards constant so that no card provides a clearly better or worse ratio. This is important for the meta game (i.e., how the players choose and optimize their card decks) to have variety, i.e., all cards to have approximately similar use rates. If there were clearly superior cards, every player would use them and the meta would become boring.

To investigate this, let's plot some card stats with respect to the elixir cost and their correlation coefficients. A correlation of 1 would mean that the stat in question is directly proportional to cost. Correlation of 0 indicates no relation, and negative correlations indicate inverse proportionality.

In [ ]:
subset=getCleanSubset(data,["Health (+Shield)","Cost"])
subset.plot.scatter(x="Cost",y="Health (+Shield)")
print("Correlation:",subset["Cost"].corr(subset["Health (+Shield)"]))

###Exercise: Do the same with some other variables such as cost vs. damage per second.

---



Add a new code cell below this text, copy-paste the code above to it, and edit it. Which values are highly correlated with the cost?

Optional: Annotate the scatterplot with card names. See, for example, https://stackoverflow.com/questions/15910019/annotate-data-points-while-plotting-from-pandas-dataframe/15911372#15911372

####Solution (expand to show code)

In [ ]:
subset=getCleanSubset(data,["Damage per second","Cost"])
subset.plot.scatter(x="Cost",y="Damage per second")
print("Correlation:",subset["Cost"].corr(subset["Damage per second"]))

###Calculating benefit based on multiple values
The code below shows how to perform math on Pandas dataframe columns and add the results as a new column. We call this column "Benefit" and calculate it as the sum of damage per second and health. This yields a tiny bit better correlation (to be improved in the exercise that follows).

In [ ]:
#Example of adding a new column that combines existing ones
s=getCleanSubset(data,["Damage per second","Health (+Shield)","Cost"])
s["Benefit"]=s["Damage per second"]+s["Health (+Shield)"]
s.plot.scatter(x="Cost",y="Benefit")
print("Correlation:",s["Cost"].corr(s["Benefit"]))

###Exercise: Improve the benefit calculation
Improve the code above by figuring out how to combine at least three stats.

How close to 1 can you get the correlation? If you can get a correlation of 1, you have reverse-engineered how Clash Royale designers balance the stats.


####Hint: (expand this section to show)


Try using "Damage per second", "Count", and "Health (+Shield)". Note that you can use all basic math operations such as multiplication and addition. Consider that the total damage caused by a troop depends on both damage per second and the troop's lifespan, and the lifespan depends on health.

####Solution: (expand this section to show)

In this example solution (which is not the only possible one), we multiply the DPS, count, and health together. This way, the benefit is calculated as the total damage the a troop (or a swarm of "Count" units) might deal to an opponent.

Of course, this is still a simple approximation, omitting details like a swarm being weak against splash damage.

In [ ]:
s=getCleanSubset(data,["Damage per second","Count","Health (+Shield)","Cost"])
s["Benefit"]=s["Count"]*s["Damage per second"]*s["Health (+Shield)"]
s.plot.scatter(x="Cost",y="Benefit")
print("Correlation:",s["Cost"].corr(s["Benefit"]))

###Linear models

Above, we've only used scatterplots and correlation. These can indicate which parameters are good for predicting benefit linearly based on cost. However, these don't directly answer questions like **"If I have a new card with these stats, what should its elixir cost be?"** or "What would be reasonable values for some stats if other stats are known?".

To answer such questions, one can try a linear model from the Scikit Learn package. The linear model is of form ```ax+b=y```. In our case, x is benefit and y is cost, and model estimates the a and b.

Note: To allow modeling, we need to first convert the Pandas dataframe columns to Numpy arrays. You can ignore the details of how this is done in the convert() function, but if you'd like to learn about Numpy arrays and reshaping, see https://numpy.org/doc/stable/user/quickstart.html

In [ ]:
#Import the tool we need
from sklearn.linear_model import LinearRegression
#A converter function that accepts Pandas dataframes and
#plain floating point or integer numbers and returns 2D Numpy arrays
#which are expected by sklearn
def convert(x):
  if isinstance(x,pd.DataFrame) or isinstance(x,pd.Series):
    return x.to_numpy().reshape([-1,1])
  elif isinstance(x,float) or isinstance(x,int):
    return np.array(x).reshape([-1,1])
  else:
    print("Can't convert this data type")
#Calculate benefit as above
s=getCleanSubset(data,["Damage per second","Count","Health (+Shield)","Cost"])
s["Benefit"]=s["Count"]*s["Damage per second"]*s["Health (+Shield)"]
#Create a new model
model=LinearRegression()
#Fit the model to data so that it predicts cost from benefit.
model.fit(convert(s["Benefit"]),convert(s["Cost"]))
#Print the regression coefficients a,b
print("a,b",model.coef_,model.intercept_)
#Now, we can predict the cost for some new benefit value.
s["PredictedCost"]=model.predict(convert(s["Benefit"]))
#Plot both actual and predicted costs
s.plot.scatter(x="Cost",y="Benefit")
s.plot.scatter(x="PredictedCost",y="Benefit")

To better compare two types of data, it's often good to plot them in the same figure. This can be done by first calling ```plt.figure()``` and then calling ```plt.scatter()``` for each dataset. This automatically results in a different color for each dataset. Finally, calling ```plt.legend()``` makes the data labels visible.

In [ ]:
# @title
plt.figure()
plt.scatter(s["PredictedCost"],s["Benefit"],label="Predicted cost")
plt.scatter(s["Cost"],s["Benefit"],label="Actual cost")
plt.xlabel("Cost")
plt.ylabel("Benefit")
plt.legend()

###Exercise: use the model to calculate and print elixir cost for a hypothetical new card with some DPS, health, and count values.

Note: after you've run the code above, the variables are also available in all the cells below it, so you can directly use ```model.predict(benefit)```. Also remember to use the convert() function.


####Solution (expand to show):

In [ ]:
#Define the parameters
dps=1000
health=100
count=2
#Compute the benefit similar to above
benefit=dps*health*count
#Print the predicted cost
print("Predicted elixir cost",model.predict(convert(benefit)))

###Exercise: find cards that violate the model. What do these tell about the model?

Basically, the cards that violate the model should give you insights of which mechanics and dynamics (e.g., card vs. card interactions) your model is not capturing.

####Hint (expand to show)


For example, you can calculate the absolute value of the cost prediction error for each card as ```np.abs(s["PredictedCost"]-s["Cost"])```. Then, sort the dataframe based on this (google Pandas sort) and print the head (see the beginning of this notebook).

Another way could be to calculate cost/benefit ratios and sort the cards based on it. Cards with very low or high ratio violate your model.

####Solution (expand to show)

The code below shows that the 5 cards with highest cost prediction error are Three Musketters, Lumberjack, Witch, Battle Ram, and Skeletons. This is not very surprising: The model does not take into account special abilities like Lumberjack's rage and the Witch spawning skeletons. Three musketeers and skeletons are swarm troops which have some additional dynamics like being able to be split onto two attack lanes. Battle ram's benefit is zero, meaning that there's probably an error in the data.  

In [ ]:
s["PredictionErrors"]=s["PredictedCost"]-s["Cost"]
s["AbsPredictionErrors"]=np.abs(s["PredictionErrors"])
sorted=s.sort_values(by="AbsPredictionErrors",ascending=False)
sorted.head()

###Exercise: Calculate health of a hypothetical new card based on some dps, count, and cost values

####Hint (expand to show)

The linear model gives us a and b for the equation ax+b=y. The y is the cost and x is the benefit. You can substitute the benefit formula used above into ax+b=y then solve any variable from others through basic algebraic manipulation.

####Solution (expand to show)

In [ ]:
#Define the known parameters
dps=1000
count=2
cost=4
#Solve the following for health: dps*count*health*a+b=cost
#=> health=(cost-b)/(dps*count*a)
#The a and b are given by the model as above
a=model.coef_
b=model.intercept_
#Print the predicted health
health=(cost-b)/(dps*count*a)
print("Predicted health",health)

Try different values! What happens to health, for example, if you increase dps or count?

###Advanced exercise 1:
The dataset we use is old and does not include all the cards of the current Clash Royale version. Write Python code that scrapes the current stats from a website like https://clashroyale.fandom.com/wiki/Cards



###Advanced exercise 2:

Implement a simulation of card vs. card battles. Use the simulation to analyze intransitive balance, i.e., "rock paper scissors" balance: Every card should win in some situations but also have one or multiple counters. Write a script that highlights which cards violate this rule. Such a script could be valuable for automatically testing that the balance is not broken.

###Advanced exercise 3:
Extend the benefit calculation to all card types. You may need to loop over all cards in a for loop and do the benefit calculation differently for each type. Also try to incorporate mechanics like death damage and ranged attacks.

How close to 1 can you get the correlation?